大纲
-
1. 抓取目标
2. 抓取分析
3. 抓取首页
4. 正则提取
5. 分页爬取
6. 运行结果

### 1. 抓取目标
提取出猫眼电影 TOP100 的`电影排名、名称、时间、主演、评分、图片`等信息，提取的站点 URL 为 http://maoyan.com/board/4。

### 2. 抓取分析
查看页面元素和URL变化规律

### 3. 抓取首页

In [17]:
import requests

def get_one_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    }

    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        return res.text
    else:
        return None
    
def main():  
    url = 'http://maoyan.com/board/4'  
    html = get_one_page(url)  
    print(html)  

main()

<!DOCTYPE html><html lang=en><head><meta charset=utf-8><link href=//p0.meituan.net rel=dns-prefetch><link href=//p1.meituan.net rel=dns-prefetch><link href=//s3plus.meituan.net rel=dns-prefetch><title>猫眼验证中心</title><meta name=keywords content="猫眼验证中心"><meta name=description content="猫眼验证中心"><meta name=applicable-device content=mobile><meta name=viewport content="width=device-width,initial-scale=1,user-scalable=no"><meta name=format-detection content="telephone=no"><meta name=format-detection content="address=no"><meta name=for content=maoyan.com><link rel=icon href=https://e.pipi.cn/overseas/faviconmy.ico type=image/x-icon><script src=https://e.pipi.cn/overseas/polyfill.min.js></script><script>var AppData= {"redirectURL":"http%3A%2F%2Fmaoyan.com%2Fboard%2F4","domain":"maoyan.com","project":"verify","env":"prod"}</script><script crossorigin=anonymous src=//www.dpfile.com/app/owl/static/owl_latest.js></script><script>"use strict";!function(){var e=0<arguments.length&&void 0!==arguments[0

### 4. 正则提取
> 一般不建议在浏览器的`Elemnets`标签页查看页面元素，因为这里看到的元素可能是渲染过的，跟我们上面直接提取的不一样，可以在源代码或请求返回查看

根据页面信息分层取提取
```
# 排名
<dd>.*?board-index.*?>(.*?)</i>
# 图片
<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)"
# 名字
<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>
# 主演、时间、评分
<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>
```

In [23]:
import re

def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(.*?)</i>.*?data-src="(.*?)".*?name.*?a.*?>(.*?)</a>.*?star.*?>(.*?)</p>.*?releasetime.*?>(.*?)</p>.*?integer.*?>(.*?)</i>.*?fraction.*?>(.*?)</i>.*?</dd>',
        re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {'index': item[0],
            'image': item[1],
            'title': item[2].strip(),
            'actor': item[3].strip()[3:] if len(item[3]) > 3 else '',
            'time': item[4].strip()[5:] if len(item[4]) > 5 else '',
            'score': item[5].strip() + item[6].strip()}
    
def main():  
    url = 'http://maoyan.com/board/4'
    
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)

main()

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 9]><!--><html><!--<![endif]-->
<head>
  <title>TOP100榜 - 猫眼电影 - 一网打尽好电影</title>
  
  <link rel="dns-prefetch" href="//p0.meituan.net"  />
  <link rel="dns-prefetch" href="//p1.meituan.net"  />
  <link rel="dns-prefetch" href="//ms0.meituan.net" />
  <link rel="dns-prefetch" href="//s0.meituan.net" />
  <link rel="dns-prefetch" href="//ms1.meituan.net" />
  <link rel="dns-prefetch" href="//analytics.meituan.com" />
  <link rel="dns-prefetch" href="//report.meituan.com" />
  <link rel="dns-prefetch" href="//frep.meituan.com" />

  
  <meta charset="utf-8">
  <meta name="keywords" content="猫眼电影,电影排行榜,热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100">
  <meta name="description" content="猫眼电影热门榜单,包括热映口碑榜,最受期待榜,国内票房榜,北美票房榜,猫眼TOP100,多维度为用户进行选片决策">
  <meta http-equiv="cleartype" content="yes" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="renderer" content="webki

#### 5. 分页爬取

In [ ]:
if __name__ == '__main__':  
    for i in range(10):  
        main(offset=i * 10)

def main(offset):  
    url = 'http://maoyan.com/board/4?offset=' + str(offset)  
    html = get_one_page(url)  
    for item in parse_one_page(html):  
        print(item) 

In [26]:
# 完整代码
import json 
import requests  
from requests.exceptions import RequestException  
import re  
import time  

def get_one_page(url):  
    try:  
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
        }
        response = requests.get(url, headers=headers)  
        if response.status_code == 200:  
            return response.text  
        return None  
    except RequestException:  
        return None  

def parse_one_page(html):  
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a' 
+ '.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>' 
+ '.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {'index': item[0],  
            'image': item[1],  
            'title': item[2],
            'actor': item[3].strip()[3:],
            'time': item[4].strip()[5:],
            'score': item[5] + item[6]
        }

def main(offset):  
    url = 'http://maoyan.com/board/4?offset=' + str(offset)  
    html = get_one_page(url)  
    for item in parse_one_page(html):  
        print(item)  

if __name__ == '__main__':  
    for i in range(1):  
        main(offset=i * 10)  
        time.sleep(1)